In [1]:
import json
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.metrics import f1_score

from sys import path

path.append('../utils/')

from utils import get_datasets

from utils import load_preds

from scipy.stats import t as table_t

In [2]:
DATASETS = ["20ng", "acm", "webkb", "reut"]
DATASETS = ["webkb"]

ALGORITHMS = ["centroide", "gbm", "knn", "lr", "rf", "svm", "bert"]

In [3]:
pd_datasets = get_datasets(DATASETS)

In [18]:
fold_values = {}
for dset in DATASETS:
    diffs = []
    alpha = 0.05 / 2
    fold_values[dset] = []
    for fold in np.arange(10):

        ppath = f"/home/welton/data/stacking/stacking_output/{dset}/10_folds/logistic_regression/proba/fold_{fold}/scoring.json"
        with open(ppath, 'r') as fd:
            pf1 = json.load(fd)["f1_macro"]
        
        mpath = f"/home/welton/data/stacking/stacking_output/{dset}/10_folds/logistic_regression/upper_train/fold_{fold}/scoring.json"
        with open(mpath, 'r') as fd:
            mf1 = json.load(fd)["f1_macro"]
        
        diffs.append(mf1 - pf1)
        fold_values[dset].append([mf1, pf1])
        print(mf1)
        
    mean = np.mean(diffs)
    std = np.std(diffs) / np.sqrt(len(diffs))

    t = table_t.ppf(1 - alpha, df=9)
    ground = np.round(mean - t * std, decimals=4)
    ceiling = np.round(mean + t * std, decimals=4)

    print(f"{dset.upper()} - ({ground} - {ceiling})")

0.8386708185116529
0.7878218424990343
0.7878105738440456
0.7811509357637301
0.7926481451006163
0.7986572639672297
0.8052624346362672
0.8000905013103361
0.8435663282721678
0.814422463835648
WEBKB - (-0.0457 - -0.0173)


In [19]:
for dset in DATASETS:
    df = pd.DataFrame(fold_values[dset], columns=["mreps", "probs"])
    mrep = np.round(np.mean(df.mreps.values) * 100, decimals=2)
    prob = np.round(np.mean(df.probs.values) * 100, decimals=2)
    print(f"{dset.upper()} - Mix Reps: {mrep} Prob: {prob}")

WEBKB - Mix Reps: 80.5 Prob: 83.65
